In [4]:
from getTexts import *
import pandas as pd 
import re 

def getTexts(folder,search):
    texts = {}
    for file in os.listdir(folder):
        if 'NOTES' not in file: 
            name = file.split('.')[0]
            if name in search: 
                path = os.path.join(folder,file)
                with open(path,'r') as file: 
                    data = file.readlines()
                if len(data) != 0: 
                    texts[name] = data[0]
    return texts

In [2]:
csv_data = pd.read_csv('/Users/amycweng/Digital Humanities/sermons.csv')
tcpIDs = [ _ for _ in csv_data['id']]

In [ ]:
charity = pd.read_csv('/Users/amycweng/Digital Humanities/Early-Modern-London/Relevant_Metadata/charityTCP.csv')
for idx, keyword in enumerate(charity['keywords']):
    if charity['id'][idx] in tcpIDs: 
        print(keyword)

In [5]:
sermons = getTexts('/Users/amycweng/Digital Humanities/charityTXT',tcpIDs)
print(len(sermons))

70


In [94]:
groups = {0: 'charity|charitable', 
        1: 'beneficience|bequeath|bestow|liberality|benevolence|goodwill| alm | almes ',
        2: 'poor|poverty|rich|wealthy|wealth|beggar|needy|lazarus|impotent|bankrup|penury| mean |meanness|prosperity',
        3: 'rate|usury|usurer|money|commodity|talon|talent|bullion|exchange|shilling|farthing| pound |penny|purchase',
        4: 'credit|creditor|loan|lend',
        5: ' debt|borrow|pledge|pawn',
        6: 'industry|industrious|thrift|labor|labour|occupation|profit|profitable|commodious',
        7: 'prison|imprisonment',
        8: 'hospital|almshouse'
        }

In [ ]:
for tcpID, t in sermons.items():
    count = 0 
    t = sermons[tcpID] 
    t = t.split(' ')
    for idx, word in enumerate(t):
        if re.match(rf'{groups[8]}',word):  
            count += 1 
    print(count)


In [82]:
tcpID = 'A01539'
t = sermons[tcpID] 
t = t.split(' ')
for idx, word in enumerate(t):
    if re.match(rf'{groups[0]}|{groups[1]}|{groups[2]}|{groups[10]}|{groups[4]}|{groups[3]}|{groups[5]}|{groups[6]}|{groups[7]}|{groups[8]}|{groups[9]}',word):  
        print(tcpID, ' '.join(t[idx-10:idx+10]))


A01539 end now what great joy can there be to a poor prisoner a condemn person that lie in hourly expectation
A01539 now what great joy can there be to a poor prisoner a condemn person that lie in hourly expectation of
A01539 and what great joy than can there be to a poor distress soul especial have lie some space of time
A01539 intolerable than death though he revel and swagger in the prison and strive to pass away the time with his
A01539 already f he be in this world as in god prison whence there can be no escape he be there
A01539 and permission the liberty of some part of this his prison yet he can never be true joyful never hearty
A01539 with more comfort or have more joy of his worldly wealth and estate than the godly man may worldly man
A01539 himself whereas the godly man have they o as favour bestow on he by god as effect and fruit of
A01539 a man more than the gift and a small matter bestow on a man out of grace and favour by
A01539 exhortation and incitement to godliness an

In [ ]:
'''
Extract the entire body text from an XML document.  
'''
outputfolder = 'charityTXT'
convert(tcpIDs,outputfolder)
print(underscores)
convert(underscores,outputfolder)

'''
TCP charity texts that are not in EP: 
'B33867', 'B13871', 'B13868', 'B13878', 'B13862', 'B13860', 
'B09696', 'A86972', 'A19579', 'A09957'
'''

In [25]:
# patterns = ["Preach't at (.*?)[^\w\s\-]",'Preached at (.*?)[^\w\s\-]','preached at (.*?)[^\w\s\-]','preached in (.*?)[^\w\s\-\']']
patterns1 = ["preach't (.*?)$","preached at (.*?)$","preached in (.*?)$","preacht (.*?)$","before the (.*?)$", "before his (.*?)$"]
patterns2 = ["preacher at (.*?)$","preacher in (.*?)$","pastor at (.*?)$","pastor of (.*?)$"]
places = []
pastorPlaces = []
import csv

outFile = open(f'sermons_locations.csv','a+')
columns = ['id','title','author','date','location']
writer = csv.DictWriter(outFile, fieldnames=columns)
writer.writeheader()

located = []
for idx, title in enumerate(csv_data['title']):
    foundPattern1 = False
    for pattern in patterns1: 
        place = re.search(rf'{pattern}', title)
        if place: 
            # places.append(place.group())
            dict = {'id': tcpIDs[idx], 
                    'title': title,
                    'author': csv_data['author'][idx],
                    'date': csv_data['date'][idx],
                    'location': place.group()
                    }
            located.append(tcpIDs[idx])
            writer.writerow(dict)
            # print(f'{tcpIDs[idx]}: {place.group()}')
            foundPattern1 = True
            break
    if not foundPattern1: 
        for pattern in patterns2: 
            place = re.search(rf'{pattern}', title)
            if place: 
                dict = {'id': tcpIDs[idx], 
                    'title': title,
                    'author': csv_data['author'][idx],
                    'date': csv_data['date'][idx],
                    'location': place.group()
                }
                # pastorPlaces.append(place.group())
                located.append(tcpIDs[idx])
                writer.writerow(dict)
                foundPattern2 = True
                break

for idx, tcpID in enumerate(tcpIDs):
    if tcpID in located: 
        continue
    dict = {'id': tcpID, 
            'title': csv_data['title'][idx],
            'author': csv_data['author'][idx],
            'date': csv_data['date'][idx],
            'location': ''
            }
    writer.writerow(dict)

# print(len(places), len(pastorPlaces), len(places)+len(pastorPlaces))
# print(places)
# print(pastorPlaces)

In [69]:
def findTextTCP(id):
    if re.match('B1|B4',id[0:2]):
        path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    else: 
        if f'{id}.P4.xml' in os.listdir(f'{TCP}/P1{id[0:2]}'):
            path = f'{TCP}/P1{id[0:2]}/{id}.P4.xml'
        elif f'{id}.P4.xml' in os.listdir(f'{TCP}/P2{id[0:2]}'): 
            path = f'{TCP}/P2{id[0:2]}/{id}.P4.xml'
    return path 

In [90]:
'''
Extracting title pages directly from TCP. 
Title pages often contain information about where the sermon was preached. 
'''
import csv 
folder = f'/Users/amycweng/Digital Humanities/'
outfile = open(f'{folder}/sermon_titles.csv','a+')
columns = ['id','title page']
writer = csv.DictWriter(outfile, fieldnames=columns)
writer.writeheader()

tcpIDs = ['A65610','A15015']
for tcpID in tcpIDs: 
    path = findTextTCP(tcpID)
    with open(path,'r') as file: 
        data = file.read()
    tag = SoupStrainer("div1", attrs={'type':'title page'})
    soup = BeautifulSoup(data,parse_only=tag,features='html.parser')
    title = []
    sentences = soup.find_all('p')
    for sentence in sentences: 
        title.append(sentence.text)
    if len(title) != 0: 
        title = ' '.join(title)
        title = re.sub('\n',' ',title)
        t = re.search(r'^(.*?) \d{4}',title)
        if t: 
            writer.writerow({'id': tcpID,  'title page': t.group()})
        else: 
            writer.writerow({'id': tcpID,  'title page': title})
outfile.close()